In [2]:
import pandas as pd
import numpy as np
import ast
from functools import reduce
from sklearn.ensemble import IsolationForest
import json
from collections import defaultdict

In [3]:
'''
given the source/checkin_checkout_history_updated.csv file, identify the users and its corresponding weeks where it has been acting differently/unusual
steps:
1. process the data to group by (user, week) and get {total visits, total length of visits, total calories burnt}
    - can use map reduce to do this, or pandas <(user, week), (total visits, total length of visits, total calories burnt)>
2. run isoloation forest on the data to identify the outliers for anomoly detection
'''

'\ngiven the source/checkin_checkout_history_updated.csv file, identify the users and its corresponding weeks where it has been acting differently/unusual\nsteps:\n1. process the data to group by (user, week) and get {total visits, total length of visits, total calories burnt}\n    - can use map reduce to do this, or pandas <(user, week), (total visits, total length of visits, total calories burnt)>\n2. run isoloation forest on the data to identify the outliers for anomoly detection\n'

In [4]:
df_byVisit = pd.read_csv('../data/_raw/checkin_checkout_history_updated.csv')
df_byVisit['checkin_time'] = pd.to_datetime(df_byVisit['checkin_time'])
df_byVisit['checkout_time'] = pd.to_datetime(df_byVisit['checkout_time'])
df_byVisit['session_seconds'] = (df_byVisit['checkout_time'] - df_byVisit['checkin_time']).dt.total_seconds()
df_byVisit['week'] = df_byVisit['checkin_time'].dt.isocalendar().week
# sometimes days in january counted as week 52 so I just changed them to week 0
df_byVisit.loc[(df_byVisit['checkin_time'].dt.month == 1) & (df_byVisit['week'] == 52), 'week'] = 0
df_byVisit.drop(['gym_id', 'checkin_time', 'checkout_time', 'workout_type'], axis=1, inplace=True)


In [18]:
CONSTANT_TOTAL_USERS = df_byVisit['user_id'].nunique()

CONSTANT_LAST_WEEK = df_byVisit['week'].max()
CONSTANT_FIRST_WEEK = df_byVisit['week'].min()

CONSTANT_AVG_CALORIES_SESSION = df_byVisit['calories_burned'].mean()
CONSTANT_AVG_LENGTH_SESSION = df_byVisit['session_seconds'].mean()

In [19]:
# The next 2 functions are the basic map-reduce to transform the data into grouping by user/week
# and counting how many times they visited in a week + total time + total calories burned

# map each entry to key=id+week and value=numVisits,length,calories
def mapFunc_groupByUserWeek(row):
    _, row_data = row
    user_id = row_data['user_id']
    calories = int(row_data['calories_burned'])
    seconds = int(row_data['session_seconds'])
    week = int(row_data['week'])
    return (str(user_id) + "-" + str(week), (1, seconds, calories))

# reduce by summing, to calculate users gym activity by week
def reduceFunc_groupByUserWeek(acc, pair):
    key, value = pair
    if key in acc:
        acc[key] = tuple(map(lambda x, y: x + y, acc[key], value))
    else:
        # i think they're already ints after the changes I made above
        acc[key] = (int(value[0]), int(value[1]), int(value[2]))
    return acc   


# if a user doesn't go in a whole week, we still need an empty entry for that week for our anomaly detection
# so I'm going to find the first week each user started going to the gym so we can add every week after

# map each entry to key=id and value=week
def mapFunc_addEmptyWeeks(row):
    _, entry = row
    key = entry['user_id']
    value = int(entry['week'])
    return (key, value)

# reduce by taking the first week they started going to the gym
def reduceFunc_findFirstWeek(acc, pair):
  
    #TODO find first and last week. make the values a tuple of (first, last)
    key, value = pair
    if key in acc:
        acc[key] = min(acc[key] , value)
    else:
        acc[key] = value
    return acc

def reduceFunc_findWeekRange(acc, pair):
    key, value = pair
    if key in acc:
        newBot = min(acc[key][0] , value)
        newTop = max(acc[key][1] , value)
        acc[key] = (newBot, newTop)
    else:
        acc[key] = (value, value)
    return acc

# this reduce function will add all the empty weeks 
# and the output will be the starting dictionary inputted to reduceFunc_groupByUserWeek
def reduceFunc_addEmptyWeeks(acc, pair):
    user, week_range = pair
    bot, top = week_range
    for i in range(int(bot), int(top) + 1):
        acc[user + '-' + str(i)] = (0, 0, 0)
    return acc

In [20]:
# setting up empty weeks first
mappedData_addEmptyWeeks = list(map(mapFunc_addEmptyWeeks, df_byVisit.iterrows()))

userlyWeekRange = reduce(reduceFunc_findWeekRange, mappedData_addEmptyWeeks, {})
emptyWeeks = reduce(reduceFunc_addEmptyWeeks, userlyWeekRange.items(), {})


In [21]:
# now adding all the actual information to the "empty weeks" dict returned from above cell
mappedData_groupByUserWeek = list(map(mapFunc_groupByUserWeek, df_byVisit.iterrows()))

weekly_userly_data = reduce(reduceFunc_groupByUserWeek, mappedData_groupByUserWeek, emptyWeeks)
weekly_userly_data_dict = dict(weekly_userly_data)



In [ ]:
# csv_df = pd.DataFrame.from_dict(reduced_data, orient='index', columns=['total_sessions', 'total_session_seconds', 'total_calories'])
# # csv_df = csv_df.reset_index()

# csv_df.to_csv('processed_data.csv', index=True)
# csv_df

# storing both dict version and other version NOTE: do not need to store in files as we can just use the variables
# with open("groupByUserWeek.json", "w") as file:
#     json.dump(weekly_userly_data, file)

# with open("groupByUserWeek_dict.json", "w") as file:
#      json.dump(weekly_userly_data_dict, file)

In [37]:
# turn jsons into a df, then run the anomoly forest on it

transformed_data = [
    (*key.split('-'), *values) for key, values in weekly_userly_data.items()
]

df_byUserWeek_noIndex = pd.DataFrame(transformed_data, columns=['user_id', 'week', 'total_sessions', 'total_seconds', 'total_calories'])
df_byUserWeek_noIndex.set_index(['user_id', 'week'], inplace=True)


# ## TODO I think I did the same thing below, not sure which you like bette
df_byUserWeek = pd.DataFrame(list(weekly_userly_data.items()), columns=["user_week", "visits_time_calories"])

df_byUserWeek[["user_id", "week"]] = df_byUserWeek["user_week"].str.split("-", expand=True)
df_byUserWeek["week"] = df_byUserWeek["week"].astype(int)

df_byUserWeek[["total_sessions", "total_seconds", "total_calories"]] = pd.DataFrame(df_byUserWeek["visits_time_calories"].tolist(), index=df_byUserWeek.index)

df_byUserWeek.drop(columns=["user_week", "visits_time_calories"], inplace=True)



In [ ]:

# df_byUserWeek_noIndex.to_json("groupByUserDataFrame.json", orient="records", indent=4)

CONSTANT_AVG_VISITS_WEEK = df_byUserWeek_noIndex['total_sessions'].mean()
CONSTANT_AVG_CALORIES_WEEK = df_byUserWeek_noIndex['total_seconds'].mean()
CONSTANT_AVG_LENGTH_WEEK = df_byUserWeek_noIndex['total_calories'].mean()

# defines the distance between 2 points where the second point is supposed to be the mean or expected value
# like the % distance away from point 2
def distance(t1, v1, c1, point2):
    v2, t2, c2 = point2
    x = (v1-v2)/v2
    y = (t1-t2)/t2
    z = (c1-c2)/c2
    return x**2+y**2+z**2



In [ ]:
# after performing anomaly detection we want to see how relevent each anomaly is
# going to use a modified TF.IDF where each document is a users list of anomaly weeks
# and each word is a week, but instead of the week appearing a lot, we're going to use distance from mean
# this will allow us to see how extreme of an anomaly it is, and if it's common among all users
# perhaps everyones gym usage went down cause there was a holiday

def mapFunc_findUserMean(row):
    key_old, value_old = row
    user, week = key_old.split('-')
    #TODO add 2 versions of week to represent min and max
    value_new = (int(week), value_old[0], value_old[1], value_old[2])
    return (user, value_new)

def reduceFunc_sumStats_weekRange(acc, pair):
    key, value = pair
    if key in acc:
        #take minimum week to find when they started, sum rest of the values
        #TODO also take max in the other part
        acc[key] = ( min(acc[key][0] , value[0]) , 
                    max(acc[key][1] , value[0]),
                    acc[key][2] + value[1] , 
                    acc[key][3] + value[2] , 
                    acc[key][4] + value[3] )
    else:
        acc[key] = (value[0], value[0], value[1], value[2], value[3])

    return acc
        
def reduceFunc_averageStats(acc, pair):
    key, value = pair
    #TODO make value also contain 
    numWeeks = value[1] - value[0] + 1
    acc[key] = (value[2] / numWeeks, value[3] / numWeeks, value[4] / numWeeks)
    return acc




In [ ]:
mappedData_findUserMean = list(map(mapFunc_findUserMean, weekly_userly_data_dict.items()))

weekly_userly_total = reduce(reduceFunc_sumStats_weekRange, mappedData_findUserMean, {})
weekly_userly_mean = reduce(reduceFunc_averageStats, weekly_userly_total.items(), {})

weekly_userly_mean_dict = dict(weekly_userly_mean)

#print(weekly_userly_mean_dict)

In [ ]:
# Okay for TF.IDF the TF part is supposed to be a number between 0-1 but distance from mean might not be
# So I think I'm going to find the max distance from the minimum box containing the data and compute the 
# max distance from any vertex to mean and divide the actual distance by that

# then we need to actually comput IDF for each week 
# which doesn't need to be repeated for each user since it's always the same

# then we need to compute TF' for each user/week

In [ ]:
df_byUserWeek["distance_from_mean"] = df_byUserWeek.apply(
    lambda row: distance(row["total_seconds"], row["total_sessions"], row["total_calories"], weekly_userly_mean[row["user_id"]]),
    axis=1
)

In [ ]:
def mapFunc_findMaxDistance(row):
    _, row_data = row
    user_id = row_data['user_id']
    distance = float(row_data['distance_from_mean'])
    return (user_id, distance)

# reduce by summing, to calculate users gym activity by week
def reduceFunc_findMaxDistance(acc, pair):
    key, value = pair
    if key in acc:
        acc[key] = max(acc[key], value)
    else:
        acc[key] = value
    return acc 




In [ ]:
#run the above map reduce
#also have dataframe modify distance row to divide by max distance per user
mappedData_findMaxDistance = list(map(mapFunc_findMaxDistance, df_byUserWeek.iterrows()))
user_max_distance = reduce(reduceFunc_findMaxDistance, mappedData_findMaxDistance, {})

df_byUserWeek["distance_from_mean"] = df_byUserWeek.apply(
    lambda row: row["distance_from_mean"] / user_max_distance[row["user_id"]],
    axis=1
)


In [ ]:
# df_byUserWeek_anomaly is the df with only anomolous data

def mapFunc_IDF(row):
    print(row)
    return (1, 2)

def reduceFunc_sumStats_weekRange(acc, pair):
    key, value = pair
    if key in acc:
        #take minimum week to find when they started, sum rest of the values
        #TODO also take max in the other part
        acc[key] = ( min(acc[key][0] , value[0]) , 
                    max(acc[key][1] , value[0]),
                    acc[key][2] + value[1] , 
                    acc[key][3] + value[2] , 
                    acc[key][4] + value[3] )
    else:
        acc[key] = (value[0], value[0], value[1], value[2], value[3])

    return acc
        
def reduceFunc_averageStats(acc, pair):
    key, value = pair
    #TODO make value also contain 
    numWeeks = value[1] - value[0] + 1
    acc[key] = (value[2] / numWeeks, value[3] / numWeeks, value[4] / numWeeks)
    return acc




In [ ]:


test = list(map(mapFunc_IDF, df_byUserWeek.iterrows()))
test

In [44]:
df_byUserWeek_noIndex_anomaly = df_byUserWeek_noIndex.copy()

model = IsolationForest(contamination=0.05, random_state=42)

df_byUserWeek_noIndex_anomaly['anomaly'] = model.fit_predict(df_byUserWeek_noIndex_anomaly)

df_byUserWeek_noIndex_anomaly = df_byUserWeek_noIndex_anomaly[df_byUserWeek_noIndex_anomaly["anomaly"] == -1]

In [ ]:
# with open("totalByUser.json", "w") as file:
#     json.dump(weekly_userly_total, file)


# with open("averageByUser.json", "w") as file:
#     json.dump(weekly_userly_mean, file)

# with open("averageByUser_dict.json", "w") as file:
#     json.dump(weekly_userly_mean_dict, file)
